#### Файл прокси xmlrpc_proxy_server_p3.ipynb на Python 3 

In [2]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.client as client

import datetime
import time
import os
import pandas as pd

server = client.ServerProxy("http://localhost:8010/RPC2")
stats = client.ServerProxy("http://localhost:8011/RPC2")

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

proxy = SimpleXMLRPCServer(("localhost", 8012), 
                            requestHandler=RequestHandler)

# Добавление строки в лог
class MyDispatcher:
    def _dispatch(self, method, args):
        print(f"Received XML-RPC request for method '{method}' with arguments {args}")
        sm = getattr(server, method)
        start_time = time.time()
        result = sm(*args)
        end_time = time.time()
        # Вычислим разницу во времени в секундах
        time_difference_seconds = end_time - start_time
        # Преобразуем разницу в миллисекунды
        time_difference_milliseconds = time_difference_seconds * 1000
        try:
            stats.add_log(method, datetime.datetime.fromtimestamp(start_time).strftime("%Y-%m-%d %H:%M:%S"), time_difference_milliseconds)
        except Exception as e:
            print("Stat server exception!")
            print(e)
        return result

proxy.register_instance(MyDispatcher())

print ("Listening on " + str(proxy.server_address))

try: 
    proxy.serve_forever()
    print('That was a nice nap.')
except KeyboardInterrupt:
    proxy.server_close()
    print('What a rude awakening!')


Listening on ('127.0.0.1', 8012)
Received XML-RPC request for method 'slice_log' with arguments ([], '2023-11-27 21:00:50', '2023-11-28 22:51:50', 1.0, 2.0)


127.0.0.1 - - [28/Nov/2023 00:25:14] "POST /RPC2 HTTP/1.1" 200 -


Received XML-RPC request for method 'slice_log' with arguments ([], '2023-11-27 21:00:50', '2023-11-28 22:51:50', 0.0, 3000.0)


127.0.0.1 - - [28/Nov/2023 00:25:53] "POST /RPC2 HTTP/1.1" 200 -


Received XML-RPC request for method 'slice_log' with arguments ([], '2023-11-27 21:00:50', '2023-11-28 22:51:50', 0.0, 3000.0)


127.0.0.1 - - [28/Nov/2023 00:26:23] "POST /RPC2 HTTP/1.1" 200 -


Received XML-RPC request for method 'slice_log' with arguments ([], '2023-11-27 21:00:50', '2023-11-28 22:51:50', 0.0, 30000.0)


127.0.0.1 - - [28/Nov/2023 00:27:01] "POST /RPC2 HTTP/1.1" 200 -


What a rude awakening!
